Hannah Smith


This data set contains the academic information of 3,046 STEM students and has 10 features: ID No, Program of Study (ProgCode), Gender, Year of Graduation (YoG), CGPA, CGPA100 (CGPA at end of first year), CGPA200 (CGPA at end of second year), CGPA300 (Cumulatove Grade Point Average at the end of the third year), CGPA400 (CGPA at end of fourth year), and SGPA (Secondary School Cumulative Grade Point Average).

Under the program of study, majors are given acroynyms for brevity. Here is the key as given by the author of the data set:

PROGRAM OF STUDY

BCH - Biochemistry

BLD - Building technology

CEN - Computer Engineering

CHE - Chemical Engineering

CHM - Industrial Chemistry

CIS - Computer Science

CVE - Civil Engineering

EEE - Electrical and Electronics Engineering

ICE - Information and Communication Engineering

MAT - Mathematics

MCB - Microbiology

MCE - Mechanical Engineering

MIS - Management and Information System

PET - Petroleum Engineering

PHYE - Industrial Physics-Electronics and IT Applications

PHYG - Industrial Physics-Applied Geophysics

PHYR - Industrial Physics-Renewable Energy'


The author is Krishnansh Verma and his data set can be found here: https://www.kaggle.com/datasets/krishnanshverma/academic-performance-of-university-student-dataset/data

In [87]:
import pandas as pd
import plotly.express as px

First, I will check the data set for any issues that need resolving before trying to find any insights.

In [88]:
df = pd.read_csv('academic_performance_dataset_V2.csv')
#Check the number of features and students
print(df.shape)

#Check every id is unique
print(len(df['ID No'].unique()))




(3046, 10)
2974


While there are 3046 records, there are only 2,974 unique IDs. Clearly, something is wrong. I'll check if there are duplicates or if the ID column is useless. 

In [89]:
duplicate_IDs =df['ID No'].duplicated()
print(df[duplicate_IDs].head(1)) #Here we can see one of the duplicated ID numbers is 76075
print(df.loc[df['ID No'] == 76075])

     ID No Prog Code Gender   YoG  CGPA  CGPA100  CGPA200  CGPA300  CGPA400  \
173  76075       MCB   Male  2014  2.32     2.61     1.98     1.77     2.67   

     SGPA  
173  2.68  
     ID No Prog Code Gender   YoG  CGPA  CGPA100  CGPA200  CGPA300  CGPA400  \
6    76075       BCH   Male  2010  3.34     3.68     3.00     3.44     3.28   
173  76075       MCB   Male  2014  2.32     2.61     1.98     1.77     2.67   

     SGPA  
6    3.02  
173  2.68  


Here we see that the two records with the same ID number are not duplicates, just mistakenly assigned the same ID. I can instead give a real ID.

In [90]:
df['ID No'] = df.index

Does a student's first semester predict what their last year GPA will be? Does their secondary school GPA predict their final GPA? Do girls have higher GPA going in as when they end?
How many students drop out? Is it affected by their program of study? Gender? Both?
Did some students graudate early? (3rd GPA == 4th year?)


Strangely enough, this data set does not appear to have null values or values that represent nulls (like 0 or negative 1). There is only one student that has a 0 value.

In [91]:
from itertools import product
#Find rows that have at least one column = 0
cols = ['YoG', 'CGPA', 'CGPA100', 'CGPA200', 'CGPA300', 'CGPA400', 'SGPA']
for col in cols:
    null_values = df[df[col] == 0]  #Filter rows where col == 0
    print(null_values)
 

Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []
Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []
Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []
Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []
Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []
      ID No Prog Code Gender   YoG  CGPA  CGPA100  CGPA200  CGPA300  CGPA400  \
2713   2713       PET   Male  2011  2.96     2.77     3.44     2.25      0.0   

      SGPA  
2713  2.81  
Empty DataFrame
Columns: [ID No, Prog Code, Gender, YoG, CGPA, CGPA100, CGPA200, CGPA300, CGPA400, SGPA]
Index: []


Possibly this student graduated in three years, or he could have just not entered his fourth year's GPA. Either way, his record was the only one that had any zeros so I couldn't get any insights out of it. I decided it would be easiest to impute this student's fourth year score. Since we have the final GPA and all the other years' GPA, it would be easy if we assumed he took on average 30 credits each year.

In [92]:
#Solve for fourth year GPA
def replace_fourth_gpa(df):
    replacement = ((df['CGPA']*30*4)-((df['CGPA100']*30)+(df['CGPA200']*30)+(df['CGPA300']*30)))/30

    #Make sure the GPA isn't invalid
    if (replacement > 5.0) or (replacement < 1):
        print("Error, GPA can't be calculated") 
        return 0

    replacement = round(replacement,2)
    print(replacement) #Out of curiousity

    return replacement

df.loc[df['CGPA400'] == 0, 'CGPA400'] = df[df['CGPA400'] == 0].apply(replace_fourth_gpa, axis=1) #apply to each row where 'CGPA400' == 0

3.38


Now that the invalid values were fixed, I wanted to see if there was any suspicious values I missed. I did this by looking at an overall distribution of GPAs.

In [93]:
px.histogram(df, x='CGPA').show()
print(f" {round(len(df.loc[df['CGPA'] < 3]) / len(df),4)*100}% of students are graduating with GPAs")

 24.72% of students are graduating with GPAs


The fact that a quarter of students had a GPA below 3.0 was deeply suscipious to me. Most internships require a 3.0 at least. I decided to filter out suspicious GPAs. Assuming each year a student took between 24 and 36 credits, I checked to see if the overall GPA could be caluclated from the yearly GPAs.

In [94]:
def gpa_checker(df):
    actual_CGPA = df['CGPA']
    gpas = [df['CGPA100'], df['CGPA200'], df['CGPA300'], df['CGPA400']]

    #If this college is like GCC, students can take between 12 and 18 credits per semester
    credit_hours = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
    combinations = list(product(credit_hours, repeat=4))

#Check all possible combinations of credit hours to see if their final GPA is possible considering their yearly GPAs
    for combo in combinations:
        sum = combo[0] +combo[1]+combo[2]+combo[3]
        possible_CGPA = round(((combo[0]*gpas[0]) + (combo[1]*gpas[1]) + (combo[2]*gpas[2]) + (combo[3]*gpas[3])) / sum,2)
        if(possible_CGPA == actual_CGPA or possible_CGPA+0.01 == actual_CGPA or possible_CGPA-0.01 == actual_CGPA):
            return True
    return False
        
        
sus_values = df[~df.apply(gpa_checker, axis=1)]
print(sus_values)

      ID No Prog Code  Gender   YoG  CGPA  CGPA100  CGPA200  CGPA300  CGPA400  \
0         0       ICE  Female  2010  3.23     2.88     3.48     2.62     2.90   
2         2       BCH    Male  2010  2.21     1.78     1.98     1.49     2.51   
3         3       BCH    Male  2010  2.70     2.67     2.44     2.00     2.35   
7         7       BCH  Female  2010  2.56     2.30     2.50     2.29     2.77   
68       68       BCH  Female  2012  2.70     2.78     2.47     2.42     2.84   
...     ...       ...     ...   ...   ...      ...      ...      ...      ...   
3027   3027      PHYG    Male  2012  3.52     3.88     3.45     2.81     3.12   
3028   3028      PHYG    Male  2014  2.46     3.27     2.22     2.94     1.92   
3031   3031      PHYG    Male  2013  2.61     3.20     2.51     2.68     2.29   
3032   3032      PHYG    Male  2014  2.15     2.96     1.67     2.45     1.92   
3035   3035      PHYG    Male  2014  2.27     2.60     2.50     2.52     1.85   

      SGPA  
0     3.13  
2

Around a third of the records were flagged as having invalid cumulative GPAs. Either this college doesn't have a 12-credit requirement per semester, or a whole bunch of students were taking above 18 credits at least one semester. It is also possible that this college has a different way of calculating GPA, or my method for checking GPAs throws out records too easily. I wanted to check to see if there was anything off about the distribution of these suspicious records.

In [ ]:
valid_records = df[~df['ID No'].isin(sus_values['ID No'])]
cols = ['CGPA', 'CGPA100', 'CGPA200', 'CGPA300', 'CGPA400', 'YoG']

# *** the following code comes from https://stackoverflow.com/questions/56727843/how-can-i-create-subplots-with-plotly-express , I just made it a method
# and edited it so it would work
import plotly.subplots as sp
def subplot_function(x_col):
    #Create figures in Express
    figure1 = px.histogram(sus_values, x=x_col, title='Invalid Records')
    figure2 = px.histogram(valid_records, x=x_col, title='Valid Records')

    # Extract traces from figures
    figure1_traces = [trace for trace in figure1["data"]]
    figure2_traces = [trace for trace in figure2["data"]]

    # Create a 1x2 subplot
    this_figure = sp.make_subplots(rows=1, cols=2, subplot_titles=['Invalid Records', 'Valid Records'])

    # Add traces to subplot
    for traces in figure1_traces:
        this_figure.add_trace(traces, row=1, col=1)
    for traces in figure2_traces:
        this_figure.add_trace(traces, row=1, col=2)

    # Show the plot
    this_figure.show()

for col in cols:
    subplot_function(col)

#*** Everything after this is 100% me again

For years 1, 2, and 3, the invalid records do seem to have slightly worse GPA than the valid records. There seems to be more correlation among year: Half of all records where the year of graudation was 2010 was flagged. In the end, there is nothing that jumps out as to what might be the reason for all the strange GPAs. A third of the database is too large to drop and would affect later insights, so I am choosing to ignore them.